In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score as accu_score

torch.manual_seed(0)

In [ ]:
data_frame = pd.read_csv("diabetes.csv")
data_frame.head()

In [ ]:
col_x = data_frame.iloc[:,:-1]
# Getting just the outcome column 
col_y = data_frame.iloc[:,-1]   

# print(col_x)
# print(col_y)

col_x = data_frame.drop('Outcome',axis=1).values
col_y = data_frame['Outcome'].values

# print()
# print(col_x)
# print(col_y)

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(col_x)

In [ ]:
X_Train, X_Vali, Y_Train, Y_Vali = train_test_split(scaled_data,col_y,test_size = 0.2, random_state= 0, shuffle = False)

X_Train = torch.FloatTensor(X_Train)
X_Vali = torch.FloatTensor(X_Vali)
Y_Train = torch.LongTensor(Y_Train)
Y_Vali = torch.LongTensor(Y_Vali)

In [ ]:
class Perceptron(nn.Module):
    def __init__(self,input_feat,output_feat):
        super().__init__()
        self.L1 = nn.Linear(input_feat, 128,2)  
        #self.BN1 = nn.BatchNorm1d(128)
        self.L1_a = nn.Linear(128,64)
        #self.BN1a = nn.BatchNorm1d(64)
        #self.L2 = nn.Linear(128,64)
        #self.BN2 = nn.BatchNorm1d(64)
        #self.L3 = nn.Linear(64,32)
        #self.BN3 = nn.BatchNorm1d(32)
        self.L4 = nn.Linear(64,output_feat)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        

    def forward(self,n_value):
        p_out = self.L1(n_value)
        p_out = self.relu(p_out)
        p_out = self.L1_a(p_out)
        #p_out = self.relu(p_out)
        #p_out = self.L2(p_out)
        #p_out = self.relu(p_out)
        #p_out = self.L3(p_out)
        #p_out = self.relu(p_out)
        p_out = self.L4(p_out)
        p_out = self.sigmoid(p_out)
        return p_out

In [ ]:
store_results = [] 
learn_rates = [0.001,0.005, 0.01, 0.05, 0.1, 0.5 ]

for lr in learn_rates:
    perceptron_model = Perceptron(8,2)
    #loss_func = nn.CrossEntropyLoss(torch.tensor([float(200/768) , float(568/768)]))
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(perceptron_model.parameters(), lr=lr)
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    epochs = 500  #280
    final_loss = []
    batch_size = 128
    

    for epoch in range(epochs):
    # # Randomly shuffle the training data (if desired) to create mini-batches
        permutatn = torch.randperm(X_Train.size()[0])

        for x in range(0, X_Train.size()[0], batch_size):
            indice = permutatn[x:x+batch_size]
            XBatch = X_Train[indice]
            YBatch = Y_Train[indice]
            
            ##forward passing to make predictions
            YPred = perceptron_model.forward(XBatch)   
            ## computing the loss
            loss = loss_func(YPred, YBatch) 
            final_loss.append(loss)

            #clearing gradients
            optimizer.zero_grad() 
            ## getting gradients using backpropagation
            loss.backward() 
            # updating the model params weights
            optimizer.step() 
    
        if epoch % 10 == 0:
            print("The epoch number is: {} and the Training Loss is : {}".format(epoch, loss.item()))
        

    # calculating the training accuracy
    with torch.no_grad():
        YPred_train = perceptron_model(X_Train)
        YPred_train_class = YPred_train.argmax(dim=1)
        training_accuracy = accu_score(Y_Train, YPred_train_class)

    # calculating testing accuracy
    with torch.no_grad():
        YPred_test = perceptron_model(X_Vali)
        YPred_test_class = YPred_test.argmax(dim=1)
        testing_accuracy = accu_score(Y_Vali, YPred_test_class)

    # Append the results to the list
    store_results.append([f'{lr:.3f}', training_accuracy * 100, testing_accuracy * 100])




In [ ]:
dataframe_results = pd.DataFrame(store_results, columns=['Learning Rate', 'Training Accuracy', 'Testing Accuracy'])

# displaying results
print(dataframe_results)